In [44]:
import numpy as np
import pandas as pd
import config
import requests

import os
import json
import time

from pandas.io.json import json_normalize

from copy import deepcopy


##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [171]:
#Candidate Search

candidate = {'q':'COLLINS, SUSAN M'
            ,'api_key':config.api_key}

c = requests.get('https://api.open.fec.gov/v1/candidates/search',params=candidate).json()
c['results']['cycles']

TypeError: list indices must be integers or slices, not str

In [45]:
%env OPENFEC_API_KEY=config.api_key
import pyopenfec as api

env: OPENFEC_API_KEY=config.api_key


In [163]:
collins_id = 'C00314575'

payload = {'per_page':'100'
          ,'committee_id':collins_id
          ,'sort':'contribution_receipt_date'
          ,'api_key':config.api_key
          ,'is_individual':'true'
          ,'two_year_transaction_period':'2020'
          ,'last_index':'4040820191646013022'
          ,'last_contribution_receipt_date': '2019-01-25T00:00:00'}

r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=payload).json()
r


{'api_version': '1.0',
 'pagination': {'count': 859,
  'last_indexes': {'last_contribution_receipt_date': '2019-02-02T00:00:00',
   'last_index': '4040820191646013446'},
  'pages': 9,
  'per_page': 100},
 'results': [{'amendment_indicator': 'A',
   'amendment_indicator_desc': 'ADD',
   'back_reference_schedule_name': None,
   'back_reference_transaction_id': None,
   'candidate_first_name': None,
   'candidate_id': None,
   'candidate_last_name': None,
   'candidate_middle_name': None,
   'candidate_name': None,
   'candidate_office': None,
   'candidate_office_district': None,
   'candidate_office_full': None,
   'candidate_office_state': None,
   'candidate_office_state_full': None,
   'candidate_prefix': None,
   'candidate_suffix': None,
   'committee': {'candidate_ids': ['S6ME00159'],
    'city': 'BANGOR',
    'committee_id': 'C00314575',
    'committee_type': 'S',
    'committee_type_full': 'Senate',
    'cycle': 2020,
    'cycles': [1996,
     1998,
     2000,
     2002,
     20

9


In [135]:
data = json_normalize(r)
r['results']
df = pd.DataFrame.from_dict(r['results'])
df

,amendment_indicator,amendment_indicator_desc,back_reference_schedule_name,back_reference_transaction_id,candidate_first_name,candidate_id,candidate_last_name,candidate_middle_name,candidate_name,candidate_office,...,receipt_type_full,recipient_committee_type,report_type,report_year,schedule_type,schedule_type_full,sub_id,transaction_id,two_year_transaction_period,unused_contbr_id
0,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013039,AC1C778FAA9F947FCB5B,2020,None
1,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013082,AC89769045D39476184F,2020,None
2,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013115,A280B235C4F5B43DDA6A,2020,None
3,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013134,AFDAAA9D1244C4274AE0,2020,None
4,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013157,A9789D8329FB84C3A976,2020,None
5,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013179,AB9B3C7DCE9E748F6BC1,2020,None
6,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013225,AADD9C48ECDD44E7DBC0,2020,None
7,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013255,A6917801701764472B28,2020,None
8,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013279,A73AF2BDC237D43DBB68,2020,None
9,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q1,2019,SA,ITEMIZED RECEIPTS,4040820191646013315,A0865815980C7485AA1A,2020,None


In [136]:
#Write results to a CSV
cwd = os.getcwd()
df.to_csv(cwd+'/collins-itemized-receipts.csv')

In [ ]:
#Template for paging through results

page = 1

def getNextPage(nextpage):
    global page
    page = page + 1
    print('PAGE ', page)
    headers = {"Authorization": "Bearer " + token}
    r = requests.get(nextpage, headers=headers)
    nextpage = r.json()['pagination']['next_link']
    records = r.json()['data']
    for user in records:
        s  = pd.Series(user,index=user.keys())
        global records_df
        records_df.loc[len(records_df)] = s
    records_df.to_csv('records.csv')
    if nextpage:
        getNextPage(nextpage)